## Import

In [ ]:
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
!pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [ ]:
import sys
sys.path.insert(0, "../input/timm-efficientdet-pytorch")
sys.path.insert(0, "../input/omegaconf")
sys.path.insert(0, "../input/weightedboxesfusion")

from ensemble_boxes import *
import torch
import numpy as np
import pandas as pd
from glob import glob
from torch.utils.data import Dataset,DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import gc
from matplotlib import pyplot as plt
from effdet import get_efficientdet_config, EfficientDet, DetBenchEval
from effdet.efficientdet import HeadNet

## Test Dataset and AUgmentation

In [ ]:
def get_valid_transforms():
    return A.Compose([
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

In [ ]:
def get_convert_transforms():
    return A.Compose([
        A.Resize(1024,1024,p=1)
    ],bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        ))

In [ ]:
DATA_ROOT_PATH = '../input/dhakaai-2020/dhaka.ai test/test'

class DatasetRetriever(Dataset):

    def __init__(self, image_ids, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

## Encoder

In [ ]:
import pickle
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
pickle_in = open('../input/encoderr/encoder.pickle','rb')
le = pickle.load(pickle_in)

In [ ]:
pickle_in.close()

In [ ]:
le.transform(['minivan'])

##  Create Dataset and Loader

In [ ]:
dataset = DatasetRetriever(
    image_ids=np.array([path.split('/')[-1][:-4] for path in glob(f'{DATA_ROOT_PATH}/*.jpg')]),
    transforms=get_valid_transforms()
)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
data_loader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

## Model

In [ ]:
def load_net(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 21
    config.image_size=512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])

    del checkpoint
    gc.collect()

    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()

In [ ]:
net = load_net('../input/dai-ck1/best-checkpoint-018epoch.bin')

## Trial

In [ ]:
image , ids = next(iter(data_loader)) 

In [ ]:
images = torch.stack(image).cuda().float()

In [ ]:
with torch.no_grad():
    det = net(images , torch.tensor([1]*images.shape[0]).float().cuda())

In [ ]:
i=1

In [ ]:
boxes = det[i].detach().cpu().numpy()[:,:4]    
scores = det[i].detach().cpu().numpy()[:,4]
labels = det[i].detach().cpu().numpy()[:,5]

In [ ]:
predictions = []

In [ ]:
indexes = np.where(scores > .10)[0]
boxes = boxes[indexes]
boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
predictions.append({
    'boxes': boxes[indexes],
    'scores': scores[indexes],
    'labels':labels[indexes]
})

In [ ]:
predictions

## Make Prediction Function and WBF

In [ ]:
def make_predictions(images, score_threshold=0.22):
    images = torch.stack(images).cuda().float()
    predictions = []
    with torch.no_grad():
        det = net(images, torch.tensor([1]*images.shape[0]).float().cuda())
        for i in range(images.shape[0]):
            boxes = det[i].detach().cpu().numpy()[:,:4]    
            scores = det[i].detach().cpu().numpy()[:,4]
            labels = det[i].detach().cpu().numpy()[:,5]
            indexes = np.where(scores > score_threshold)[0]
            boxes = boxes[indexes]
            boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
            predictions.append({
                'boxes': boxes[indexes],
                'scores': scores[indexes],
                'labels':labels[indexes]
            })
    return [predictions]

In [ ]:
def run_wbf(predictions, image_index, image_size=512, iou_thr=0.5, skip_box_thr=0.12, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
    labels = [prediction[image_index]['labels'].tolist() for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

## Trial Again

In [ ]:
image , ids = next(iter(data_loader))

In [ ]:
predictions = make_predictions(image)

In [ ]:
i = 0
sample = image[i].permute(1,2,0).cpu().numpy()

In [ ]:
boxes , scores , labels = run_wbf(predictions,image_index=i)

In [ ]:
boxes

In [ ]:
boxes = boxes.astype(np.int32).clip(min=0,max=511)

In [ ]:
fig , ax = plt.subplots(1,1,figsize=(20,8))
for i,box in enumerate(boxes):
    cv2.rectangle(sample,(box[0],box[1]),(box[2],box[3]),(1,0,0),2)
    cv2.putText(sample,le.inverse_transform([int(labels[i])])[0] , (box[0], box[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 1)
    
ax.set_axis_off()
ax.imshow(sample)

In [ ]:
boxes

In [ ]:
labels

## Trial for Convert Aug

In [ ]:
conv_transform = get_convert_transforms()

In [ ]:
trial = conv_transform(image=np.float32(image[0].permute(1,2,0)),bboxes=boxes,labels=labels)

In [ ]:
img = trial['image']

In [ ]:
img

In [ ]:
trial['bboxes']

In [ ]:
bbox = (torch.stack(tuple(map(torch.tensor, zip(*trial['bboxes'])))).permute(1, 0)).numpy()

In [ ]:
fig , ax = plt.subplots(1,1,figsize=(20,8))
for i,box in enumerate(bbox):
    cv2.rectangle(img,(int(box[0]),int(box[1])),(int(box[2]),int(box[3])),(1,0,0),2)
    #cv2.putText(sample,le.inverse_transform([int(labels[i])])[0] , (box[0], box[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 1)
    
ax.set_axis_off()
ax.imshow(img)

In [ ]:
img

## Complete Pipeline to Get Prediction

In [ ]:
image , ids = next(iter(data_loader))
predictions = make_predictions(image)

i=0

sample = image[i].permute(1,2,0).cpu().numpy()

boxes , scores , labels = run_wbf(predictions,image_index=i)
boxes = boxes.astype(np.int32).clip(min=0,max=511)

trial = conv_transform(image=sample,bboxes=boxes,labels=labels)

img = trial['image']

bbox = np.int32((torch.stack(tuple(map(torch.tensor, zip(*trial['bboxes'])))).permute(1, 0)).numpy())

In [ ]:
fig , ax = plt.subplots(1,1,figsize=(20,8))
for i,box in enumerate(bbox):
    cv2.rectangle(img,(int(box[0]),int(box[1])),(int(box[2]),int(box[3])),(1,0,0),2)
    cv2.putText(img,le.inverse_transform([int(labels[i])])[0] , (int(box[0]), int(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
ax.set_axis_off()
ax.imshow(img)

## Make Submission.csv

In [ ]:
dict_try = {'image_id':[],'class':[],'score':[],'xmin':[],'ymin':[],'xmax':[],'ymax':[]}

In [ ]:
df_try = pd.DataFrame.from_dict(dict_try)

In [ ]:
image , im_id = next(iter(data_loader))
im_id

In [ ]:
predictions = make_predictions(image)

i=0

sample = image[i].permute(1,2,0).cpu().numpy()

boxes , scores , labels = run_wbf(predictions,image_index=i)
boxes = boxes.astype(np.int32).clip(min=0,max=511)

trial = conv_transform(image=sample,bboxes=boxes,labels=labels)

img = trial['image']

bbox = np.int32((torch.stack(tuple(map(torch.tensor, zip(*trial['bboxes'])))).permute(1, 0)).numpy())

In [ ]:
predictions[0][0]['scores']

In [ ]:
le.inverse_transform([int(labels[0])])[0]

In [ ]:
bbox[0][0]

In [ ]:
for i,box in enumerate(bbox):
    dict_1 = {'image_id':[f"{im_id[0]}.jpg"] ,'class':le.inverse_transform([int(labels[i])])[0] , 
              'score' : predictions[0][0]['scores'][i],
             'xmin':box[0] , 'ymin':box[1] , 'xmax':box[2] , 'ymax':box[3]}
    df_1 = pd.DataFrame.from_dict(dict_1)
    df_try = pd.concat([df_try,df_1],ignore_index=True)

In [ ]:
df_try

## Make Final Submission

In [ ]:
dict_try = {'image_id':[],'class':[],'score':[],'xmin':[],'ymin':[],'xmax':[],'ymax':[]}
df_try = pd.DataFrame.from_dict(dict_try)

In [ ]:
for image,im_id in data_loader:
    predictions = make_predictions(image)
    for k in range(len(image)):
        sample = image[k].permute(1,2,0).cpu().numpy()

        boxes , scores , labels = run_wbf(predictions,image_index=k)
        boxes = boxes.astype(np.int32).clip(min=0,max=511)

        trial = conv_transform(image=sample,bboxes=boxes,labels=labels)

        img = trial['image']
        
        try:
            bbox = np.int32((torch.stack(tuple(map(torch.tensor, zip(*trial['bboxes'])))).permute(1, 0)).numpy())
        except:
            continue
        
        for i,box in enumerate(bbox):
            dict_1 = {'image_id':[f"{im_id[k]}.jpg"] ,'class':le.inverse_transform([int(labels[i])])[0] , 
                      'score' : predictions[0][k]['scores'][i],
                     'xmin':box[0] , 'ymin':box[1] , 'xmax':box[2] , 'ymax':box[3]}
            df_1 = pd.DataFrame.from_dict(dict_1)
            df_try = pd.concat([df_try,df_1],ignore_index=True)

## Error Fixing

In [ ]:
df_try

In [ ]:
df_try.loc[:,'width'] = 1024
df_try.loc[:,'height'] = 1024

In [ ]:
df_try.to_csv('submit1.csv',index=False)

## Visualize

In [ ]:
def run_wbf(predictions, image_index, image_size=512, iou_thr=0.50, skip_box_thr=0.30, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
    labels = [prediction[image_index]['labels'].tolist() for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

In [ ]:
k = np.random.randint(len(data_loader))
for j,(image,im_id) in enumerate(data_loader):
    if (j==k):
        predictions = make_predictions(image)

        i=1

        sample = image[i].permute(1,2,0).cpu().numpy()

        boxes , scores , labels = run_wbf(predictions,image_index=i)
        boxes = boxes.astype(np.int32).clip(min=0,max=511)

        trial = conv_transform(image=sample,bboxes=boxes,labels=labels)

        img = trial['image']

        bbox = np.int32((torch.stack(tuple(map(torch.tensor, zip(*trial['bboxes'])))).permute(1, 0)).numpy())
        
        
        for i,box in enumerate(bbox):
            cv2.rectangle(img,(int(box[0]),int(box[1])),(int(box[2]),int(box[3])),(1,0,0),2)
            cv2.putText(img,le.inverse_transform([int(labels[i])])[0] , (int(box[0]), int(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,1,0), 2)
            
        break
        
fig , ax = plt.subplots(1,1,figsize=(20,8))

ax.set_axis_off()
ax.imshow(img)

## Another submission for IOU Threshold change

In [ ]:
dict_try = {'image_id':[],'class':[],'score':[],'xmin':[],'ymin':[],'xmax':[],'ymax':[]}
df_try = pd.DataFrame.from_dict(dict_try)

In [ ]:
for image,im_id in data_loader:
    predictions = make_predictions(image)
    for k in range(len(image)):
        sample = image[k].permute(1,2,0).cpu().numpy()

        boxes , scores , labels = run_wbf(predictions,image_index=k)
        boxes = boxes.astype(np.int32).clip(min=0,max=511)

        trial = conv_transform(image=sample,bboxes=boxes,labels=labels)

        img = trial['image']
        
        try:
            bbox = np.int32((torch.stack(tuple(map(torch.tensor, zip(*trial['bboxes'])))).permute(1, 0)).numpy())
        except:
            continue
        
        for i,box in enumerate(bbox):
            dict_1 = {'image_id':[f"{im_id[k]}.jpg"] ,'class':le.inverse_transform([int(labels[i])])[0] , 
                      'score' : predictions[0][k]['scores'][i],
                     'xmin':box[0] , 'ymin':box[1] , 'xmax':box[2] , 'ymax':box[3]}
            df_1 = pd.DataFrame.from_dict(dict_1)
            df_try = pd.concat([df_try,df_1],ignore_index=True)

In [ ]:
df_try.loc[:,'width'] = 1024
df_try.loc[:,'height'] = 1024

In [ ]:
df_try.to_csv('submit1_2.csv',index=False)